Load the json dictionary

In [1]:
import json

In [2]:
from matplotlib import pyplot as plt

In [3]:
import cv2

In [4]:
import os
import random
import colorsys

In [5]:
with open('./data2.json', 'r') as json_file:
    json_dict = json.load(json_file)
json_file.close()

In `labels` folder, the structure have to as same as the `images` folder

In [6]:
def xPixelFromCoords(x,y,z):
    kRealWorldXtoZ = 1.122133
    kResX = 640
    fCoeffX = kResX / kRealWorldXtoZ
    return int((fCoeffX * x / z) + (kResX / 2))
#     return int(156.8584456124928 + 0.0976862095248 * x - 0.0006444357104 * y + 0.0015715946682 * z)

In [7]:
def yPixelFromCoords(x,y,z):
    kRealWorldYtoZ = 0.84176
    kResY = 480
    fCoeffY = kResY / kRealWorldYtoZ
    return int((kResY / 2) - (fCoeffY * y / z))
#     return int(125.5357201011431 + 0.0002153447766 * x - 0.1184874093530 * y - 0.0022134485957 * z)

In [8]:
def vis_bbox(img, bboxes, width, height, label):
    num_classes = len(label)
    hsv_tuples = [(1.0 * x / num_classes, 1., 1.) for x in range(num_classes)]
    colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
    colors = list(map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)), colors))

    random.seed(0)
    random.shuffle(colors)
    random.seed(None)
    img = img * 255
    for i, bbox in enumerate(bboxes):
        xc = int(bbox[1] * width)
        yc = int(bbox[2] * height)
        w = int(bbox[3] * width / 2)
        h = int(bbox[4] * height / 2)
        c1 = (xc - w, yc - h)
        c2 = (xc + w, yc + h)
        class_ind = bbox[0]
        bbox_color = colors[class_ind]
        bbox_thick = int(0.6 * (height + width) / 300)
        cv2.rectangle(img, c1, c2, bbox_color, bbox_thick)
#         print(bbox[0],c1,c2)
    return img

  Joint number -> Joint name \
     1 -> HEAD \
     2 -> NECK \
     3 -> TORSO \
     4 -> LEFT_SHOULDER \
     5 -> LEFT_ELBOW \
     6 -> RIGHT_SHOULDER \
     7 -> RIGHT_ELBOW \
     8 -> LEFT_HIP \
     9 -> LEFT_KNEE \
    10 -> RIGHT_HIP \
    11 -> RIGHT_KNEE \
    12 -> LEFT_HAND \
    13 -> RIGHT_HAND \
    14 -> LEFT_FOOT \
    15 -> RIGHT_FOOT \
 
 In json file the keypoint begin at 0

In [9]:
joint_dict = {"0":"HEAD","1":"NECK","2":"TORSO","3":"LEFT_SHOULDER",
              "4":"LEFT_ELBOW","5":"RIGHT_SHOULDER","6":"RIGHT_ELBOW",
              "7":"LEFT_HIP","8":"LEFT_KNEE","9":"RIGHT_HIP","10":"RIGHT_KNEE",
              "11":"LEFT_HAND","12":"RIGHT_HAND","13":"LEFT_FOOT","14":"RIGHT_FOOT"}
joint_bbox_dict = {"0":80,"1":60,"2":120,"3":60,
              "4":60,"5":60,"6":60,
              "7":50,"8":50,"9":50,"10":50,
              "11":50,"12":50,"13":50,"14":50}
#while CONF == 0, the label will shift away from body

In [10]:
keypoints_list = [0,11,12,13,14]

In [11]:
image_path_list = [] #for train.txt and valid.txt
for i, ten_int in enumerate(json_dict):
    img_folder_path = json_dict[ten_int]['img_folder_path']
    img_label_path = img_folder_path.replace("images", "labels")
    if not os.path.exists(img_label_path):
        os.makedirs(img_label_path)
    RGB_prefix = 'RGB_'
    RGB_postfix = '.png'
    for j, frameID in enumerate(json_dict[ten_int]['frames']):
        image_path = img_folder_path + '/' + RGB_prefix + frameID + RGB_postfix
        image_path_list.append(image_path)
        img = plt.imread(image_path)
        img_width = img.shape[1] #default as 640
        img_height = img.shape[0] # default as 480
        joints_dict = json_dict[ten_int]['frames'][frameID]['P']
        #create a txt file 
        txt_file = img_label_path + '/' + RGB_prefix + frameID + '.txt'
        f = open(txt_file, 'w')
        vis_bboxes = []
        for t, jointID in enumerate(joints_dict):
            if int(jointID) in keypoints_list:
                coor = joints_dict[jointID]['coor']
                CONF = joints_dict[jointID]['CONF']
                if CONF:
                    x = coor[0]
                    y = coor[1]
                    z = coor[2]
                    x_pixel = xPixelFromCoords(x,y,z) #x_c
                    y_pixel = yPixelFromCoords(x,y,z) #y_c
                    if x_pixel <= img_width and y_pixel <= img_height:
        #                 cls = int(jointID) - 1
                        cls = keypoints_list.index(int(jointID))
                        x_c = x_pixel / img_width
                        y_c = y_pixel / img_height
                        w = joint_bbox_dict[jointID] / img_width
                        h = joint_bbox_dict[jointID] / img_height
                        vis_bboxes.append([cls, x_c, y_c, w, h])
                        #write to a label txt
                        wt = str(cls)+" "+str(x_c)+" "+str(y_c)+" "+str(w)+" "+str(h)+"\r"
                        f.write(wt)
        f.close()
#         pic = vis_bbox(img, vis_bboxes, img_width, img_height, keypoints_list)      
#         pic = pic / 255
#         plt.imshow(pic)
#         plt.savefig('./vis/'+frameID+'.png')
#         plt.show()

KeyboardInterrupt: 

In [ ]:
image_path_list